In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
import malaya_speech.train.model.cnn_rnn as cnn_rnn
import tensorflow as tf


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
config = malaya_speech.config.speakernet_featurizer_config
featurizer = malaya_speech.featurization.SpeakerNetFeaturizer(config)

In [5]:
y, sr = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav', sr = 16000)
y1, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav', sr = 16000)
len(y), len(y1)

(90090, 56298)

In [6]:
%%time
featurizer.vectorize(y).shape

CPU times: user 12.1 ms, sys: 3.86 ms, total: 16 ms
Wall time: 9.39 ms


(564, 64)

In [7]:
v = tf.placeholder(tf.float32, [None, None, 64])
v_length = tf.placeholder(tf.int32, [None])

In [8]:
model = cnn_rnn.Model(v)
model.logits

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


<tf.Tensor 'batch_normalization_8/batchnorm/add_1:0' shape=(?, ?, 1024) dtype=float32>

In [9]:
ctc_layer = tf.layers.dense(model.logits, malaya_speech.char.VOCAB_SIZE)
ctc_layer

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


<tf.Tensor 'dense/BiasAdd:0' shape=(?, ?, 256) dtype=float32>

In [10]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [11]:
padded = malaya_speech.padding.sequence_nd([featurizer.vectorize(y), featurizer.vectorize(y1)], dim = 0)

In [12]:
sess.run(ctc_layer, feed_dict = {v: padded}).shape

(2, 156, 256)